In [1]:
#import libraries
import pandas as pd
from ast import literal_eval
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report
import numpy as np

So far this dataframe isn't being used. But I am keeping it here for reference until a final decision is made on storing the numpy arrays.

In [2]:
#list of columns in our 'movies.csv' that need to be read in using literal_eval
#to ensure they are represented as vectors rather than strings
converterList = ['imdb_genres', 'tmdb_genres', 'binary_tmdb', 'binary_imdb',
                 'imdb_clean', 'tmdb_clean',
               'imdb_w2v_plot', 'tmdb_w2v_plot', 'tmdb_bow_plot', 'imdb_bow_plot']

converterDict = {column: literal_eval for column in converterList}

movies = pd.read_csv('data/movies.csv', encoding='utf-8',
                     converters=converterDict)

In [5]:
movies.head()

,tmdb_id,imdb_id,tmdb_genres,imdb_genres,binary_tmdb,binary_imdb,tmdb_plot,imdb_plot,popularity,release_date,title,vote_average,vote_count,tmdb_clean_plot,imdb_clean_plot,tmdb_w2v_plot,imdb_w2v_plot,tmdb_bow_plot,imdb_bow_plot
0,278,tt0111161,"[18, 80]","[80, 18]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",Framed in the 1940s for the double murder of h...,Chronicles the experiences of a formerly succe...,28.527767,1994-09-23,The Shawshank Redemption,8.5,9773,"['framed', '1940s', 'double', 'murder', 'wife'...","['chronicles', 'experiences', 'formerly', 'suc...","[0.014165705069899559, 0.035729147493839264, 0...","[0.004663567990064621, 0.09018586575984955, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,238,tt0068646,"[18, 80]","[80, 18]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",Spanning the years 1945 to 1955 a chronicle o...,When the aging head of a famous crime family d...,36.965452,1972-03-14,The Godfather,8.5,7394,"['spanning', 'years', '1945', '1955', 'chronic...","['aging', 'head', 'famous', 'crime', 'family',...","[-0.016820836812257767, 0.05966977775096893, -...","[-0.013326308690011501, 0.08134819567203522, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,424,tt0108052,"[18, 36, 10752]","[18, 36]","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",The true story of how businessman Oskar Schind...,Oskar Schindler is a vainglorious and greedy G...,19.945455,1993-11-29,Schindler's List,8.4,5518,"['true', 'story', 'businessman', 'oskar', 'sch...","['oskar', 'schindler', 'vainglorious', 'greedy...","[0.0758906751871109, 0.02254812978208065, 0.06...","[0.05338115245103836, 0.10281133651733398, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,240,tt0071562,"[18, 80]","[80, 18]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",In the continuing saga of the Corleone crime f...,The continuing saga of the Corleone crime fami...,30.191804,1974-12-20,The Godfather: Part II,8.4,4249,"['continuing', 'saga', 'corleone', 'crime', 'f...","['continuing', 'saga', 'corleone', 'crime', 'f...","[-0.05790800228714943, 0.07111673057079315, -0...","[-0.05151921883225441, 0.07896284759044647, -0...","[0.0, 0.0, 0.0, 0.34648671489524274, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,452522,tt0278784,"[18, 9648]","[80, 18, 9648, 53]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...",Standalone version of the series pilot with an...,When beautiful young Laura Palmer is found br...,5.969249,1989-12-31,Twin Peaks,8.4,123,"['standalone', 'version', 'series', 'pilot', '...","['beautiful', 'young', 'laura', 'palmer', 'fou...","[-0.05888228118419647, -0.05345569923520088, -...","[0.0026558770332485437, 0.10140948742628098, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Bag of Words

In [3]:
#load in previously saved ndarrays
binary_tmdb = np.load('data/binary_tmdb.npy')
binary_imdb = np.load('data/binary_imdb.npy')

tmdb_bow = np.load('data/tmdb_bow.npy')
imdb_bow = np.load('data/tmdb_bow.npy')

In [4]:
train_y, test_y = train_test_split(binary_tmdb, test_size=0.5, random_state=9001)
train_x, test_x = train_test_split(tmdb_bow, test_size=0.5, random_state=9001)


In [84]:
parameters = {'kernel':['linear'], 'C':[0.01, 0.1, 1.0]}
gridCV = GridSearchCV(SVC(class_weight='balanced'), parameters, scoring=make_scorer(f1_score, average='micro'))
classif = OneVsRestClassifier(gridCV)

classif.fit(train_x, train_y)

OneVsRestClassifier(estimator=GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),...refit=True, return_train_score=True,
       scoring=make_scorer(f1_score, average=micro), verbose=0),
          n_jobs=1)

In [101]:
train_yhat=classif.predict(train_x)
test_yhat=classif.predict(test_x)

#todo - I can add the target_names parameter but i don't know the order 
#       that the genres are listed

print (classification_report(train_y, train_yhat))
print('\n\n')
print (classification_report(test_y, test_yhat))

             precision    recall  f1-score   support

          0       0.91      1.00      0.95        88
          1       0.92      1.00      0.96        55
          2       0.95      1.00      0.97        52
          3       0.99      0.97      0.98       327
          4       1.00      1.00      1.00        26
          5       0.17      1.00      0.29        84
          6       0.87      1.00      0.93       121
          7       0.08      1.00      0.14        39
          8       1.00      1.00      1.00        11
          9       0.92      1.00      0.96       112
         10       0.19      1.00      0.31        93
         11       0.93      1.00      0.96        52
         12       0.93      1.00      0.96        55
         13       1.00      1.00      1.00        24
         14       0.00      0.00      0.00        77
         15       0.97      1.00      0.98        59
         16       0.95      1.00      0.97        35
         17       1.00      1.00      1.00   

C:\Users\A706GZZ\Documents\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


It appears that overfitting may be occuring. The training precision/recall is 0.76/0.93, respectively, whereas the test results are 0.42/0.51, respectively.

# Words2Vec

In [23]:
tmdb_w2v = np.load('data/tmdb_w2v.npy')
imdb_w2v = np.load('data/imdb_w2v.npy')

train_x, test_x = train_test_split(tmdb_w2v, test_size=0.5, random_state=9001)

In [27]:
parameters = {'kernel':['linear'], 'C':[0.01, 0.1, 1.0]}
#gridCV = GridSearchCV(SVC(class_weight='balanced'), parameters, scoring=make_scorer(f1_score, average='micro'))
classif = OneVsRestClassifier(gridCV)

classif.fit(train_x, train_y)

ValueError: setting an array element with a sequence.

In [28]:
np.array(train_y[:5])

array([[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

In [26]:
train_x[:5]

array([array([ 3.82862911e-02,  8.59215781e-02,  2.48293672e-02,  6.59312382e-02,
       -1.26422383e-02, -3.17117432e-04,  4.88108769e-02, -1.02024242e-01,
        5.45483045e-02,  1.04881950e-01,  5.22779394e-03, -5.19303046e-02,
        2.72857826e-02, -2.55843457e-02, -4.22449522e-02,  6.28502890e-02,
        2.56188437e-02,  9.01210606e-02,  9.21207946e-04, -3.44503634e-02,
       -1.38284434e-02,  5.39280437e-02,  8.02744980e-05,  7.63762072e-02,
        8.30290392e-02, -7.20480233e-02, -5.77206835e-02,  8.98198634e-02,
        1.05012313e-01, -4.84476499e-02,  5.16012423e-02, -1.74245425e-02,
       -4.16498594e-02,  8.41674805e-02, -5.15985899e-02, -2.88852192e-02,
        5.16092069e-02, -6.00957461e-02,  5.64707862e-03,  7.83346444e-02,
        8.59746486e-02, -1.61079736e-03,  7.59622315e-03,  3.16256657e-02,
        8.50511622e-03, -3.79505977e-02,  3.15047540e-02,  3.84727158e-02,
        2.95675527e-02, -4.14932929e-02, -4.17780653e-02,  1.20768674e-01,
       -4.83929180